In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

df_mad = pd.read_csv("..\data_sample\estat_avia_par_es_filtered_en.csv")

<>:7: SyntaxWarning: invalid escape sequence '\d'
<>:7: SyntaxWarning: invalid escape sequence '\d'
C:\Users\adria\AppData\Local\Temp\ipykernel_24936\3504604276.py:7: SyntaxWarning: invalid escape sequence '\d'
  df_mad = pd.read_csv("..\data_sample\estat_avia_par_es_filtered_en.csv")
C:\Users\adria\AppData\Local\Temp\ipykernel_24936\3504604276.py:7: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mad = pd.read_csv("..\data_sample\estat_avia_par_es_filtered_en.csv")


In [2]:
df_mad.head()

,DATAFLOW,LAST UPDATE,freq,unit,tra_meas,airp_pr,TIME_PERIOD,OBS_VALUE,OBS_FLAG,CONF_STATUS
0,ESTAT:AVIA_PAR_ES(1.0),17/02/25 23:00:00,Monthly,Passenger,Passengers carried,FUERTEVENTURA airport - WIEN-SCHWECHAT airport,2011-01,2479,NaN,NaN
1,ESTAT:AVIA_PAR_ES(1.0),17/02/25 23:00:00,Monthly,Passenger,Passengers carried,FUERTEVENTURA airport - WIEN-SCHWECHAT airport,2011-02,2264,NaN,NaN
2,ESTAT:AVIA_PAR_ES(1.0),17/02/25 23:00:00,Monthly,Passenger,Passengers carried,FUERTEVENTURA airport - WIEN-SCHWECHAT airport,2011-03,3051,NaN,NaN
3,ESTAT:AVIA_PAR_ES(1.0),17/02/25 23:00:00,Monthly,Passenger,Passengers carried,FUERTEVENTURA airport - WIEN-SCHWECHAT airport,2011-04,3523,NaN,NaN
4,ESTAT:AVIA_PAR_ES(1.0),17/02/25 23:00:00,Monthly,Passenger,Passengers carried,FUERTEVENTURA airport - WIEN-SCHWECHAT airport,2011-05,1726,NaN,NaN


In [3]:
df_mad.tail()

,DATAFLOW,LAST UPDATE,freq,unit,tra_meas,airp_pr,TIME_PERIOD,OBS_VALUE,OBS_FLAG,CONF_STATUS
160492,ESTAT:AVIA_PAR_ES(1.0),17/02/25 23:00:00,Monthly,Passenger,Passengers carried,SEVILLA airport - LONDON STANSTED airport,2024-08,11169,NaN,NaN
160493,ESTAT:AVIA_PAR_ES(1.0),17/02/25 23:00:00,Monthly,Passenger,Passengers carried,SEVILLA airport - LONDON STANSTED airport,2024-09,10319,NaN,NaN
160494,ESTAT:AVIA_PAR_ES(1.0),17/02/25 23:00:00,Monthly,Passenger,Passengers carried,SEVILLA airport - LONDON STANSTED airport,2024-10,12900,NaN,NaN
160495,ESTAT:AVIA_PAR_ES(1.0),17/02/25 23:00:00,Monthly,Passenger,Passengers carried,SEVILLA airport - LONDON STANSTED airport,2024-11,15539,NaN,NaN
160496,ESTAT:AVIA_PAR_ES(1.0),17/02/25 23:00:00,Monthly,Passenger,Passengers carried,SEVILLA airport - LONDON STANSTED airport,2024-12,14007,NaN,NaN


In [4]:
df_mad.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160497 entries, 0 to 160496
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   DATAFLOW     160497 non-null  object 
 1   LAST UPDATE  160497 non-null  object 
 2   freq         160497 non-null  object 
 3   unit         160497 non-null  object 
 4   tra_meas     160497 non-null  object 
 5   airp_pr      160497 non-null  object 
 6   TIME_PERIOD  160497 non-null  object 
 7   OBS_VALUE    160497 non-null  int64  
 8   OBS_FLAG     5 non-null       object 
 9   CONF_STATUS  0 non-null       float64
dtypes: float64(1), int64(1), object(8)
memory usage: 12.2+ MB


In [5]:
df_mad.isna().sum()

DATAFLOW            0
LAST UPDATE         0
freq                0
unit                0
tra_meas            0
airp_pr             0
TIME_PERIOD         0
OBS_VALUE           0
OBS_FLAG       160492
CONF_STATUS    160497
dtype: int64

In [6]:
df_mad.duplicated().sum()

0

In [7]:
#Vamos a elimiinar las columnas que no nos interesan. 

df_mad_clean = df_mad.drop(columns=["DATAFLOW", "LAST UPDATE", "freq", "unit", "tra_meas", "OBS_FLAG", "CONF_STATUS"])

In [8]:
#Ahora creamos las columnas Origen, Destino y filtramos las rutas que no tengan madrid como origen o destino. 
df_mad_clean[['Origen', 'Destino']] = df_mad_clean['airp_pr'].str.split(" - ", n=1, expand=True)
df_mad_clean = df_mad_clean.drop(['airp_pr'], axis=1, errors='ignore')

In [9]:
#Ahora vamos a filtrar las rutas que no tengan madrid como origen o destino.
df_mad_clean = df_mad_clean[(df_mad_clean['Origen'] == 'ADOLFO SUAREZ MADRID-BARAJAS airport') | (df_mad_clean['Destino'] == 'ADOLFO SUAREZ MADRID-BARAJAS airport')]
#Creamos la columna Tipo de trafico, que nos indica si es un vuelo de salida o de llegada.
df_mad_clean['Tipo de trafico'] = np.where(df_mad_clean['Origen'] == 'ADOLFO SUAREZ MADRID-BARAJAS airport', 'Salida', 'Llegada')

In [10]:
#Renombramos las columnas para que sean mas legibles y convertimos la columna time_period a datetime.
df_mad_final = df_mad_clean.rename(columns={"OBS_VALUE": "Pasajeros", "TIME_PERIOD": "Fecha"})
df_mad_final['Fecha'] = pd.to_datetime(df_mad_final['Fecha'], format='%Y-%m')

# Crear columnas 'Año' y 'Mes'
df_mad_final['Año'] = df_mad_final['Fecha'].dt.year
df_mad_final['Mes'] = df_mad_final['Fecha'].dt.month
#Guardamos el dataframe limpio en un nuevo csv.
df_mad_final.to_csv("..\data_sample\mad_clean.csv", index=False)


<>:9: SyntaxWarning: invalid escape sequence '\d'
<>:9: SyntaxWarning: invalid escape sequence '\d'
C:\Users\adria\AppData\Local\Temp\ipykernel_24936\966983477.py:9: SyntaxWarning: invalid escape sequence '\d'
  df_mad_final.to_csv("..\data_sample\mad_clean.csv", index=False)


In [11]:
df_mad_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25609 entries, 3386 to 156388
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Fecha            25609 non-null  datetime64[ns]
 1   Pasajeros        25609 non-null  int64         
 2   Origen           25609 non-null  object        
 3   Destino          25609 non-null  object        
 4   Tipo de trafico  25609 non-null  object        
 5   Año              25609 non-null  int32         
 6   Mes              25609 non-null  int32         
dtypes: datetime64[ns](1), int32(2), int64(1), object(3)
memory usage: 1.4+ MB


In [12]:
df_mad_final.describe()

,Fecha,Pasajeros,Año,Mes
count,25609,25609.000000,25609.000000,25609.000000
mean,2017-06-23 06:00:59.885196800,34306.432270,2017.018314,6.508337
min,2010-01-01 00:00:00,0.000000,2010.000000,1.000000
25%,2013-10-01 00:00:00,11896.000000,2013.000000,4.000000
50%,2017-06-01 00:00:00,20823.000000,2017.000000,7.000000
75%,2021-05-01 00:00:00,42890.000000,2021.000000,10.000000
max,2024-12-01 00:00:00,295603.000000,2024.000000,12.000000
std,NaN,36179.630157,4.354921,3.450120
